In [2]:
import pandas as pd
import numpy as np

In [3]:
id_mat_df = pd.read_pickle("./id_mat_matrix.pkl")
id_mat_df.iloc[:3]

,1,山东钢铁股份有限公司莱芜分公司,547-420轧机机列,下调心托辊支架,六渣池桥式起重机维修,热电车间工艺报警记录,直通截止阀,后弓子板,光面绝缘防砸鞋,西门子数字量输入输出模块(扩展）,...,电机/MF013-16000016/附图,高温覆膜除尘袋φ130*6000 ≤10mg/Nm3 600g,开关柜智能操控装置（开关状态指示仪）,螺栓M16×55,普通压制钢丝绳(加隔离套)φ15*6米,内六角螺丝 普通 英制 内 六角 3/8*16*3W,焊工面罩,"2.5""回收钢丝软管",超声波水表,妙管家洗衣液
sup_id,,,,,,,,,,,,,,,,,,,,,
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
#基于物料相似度的筛选, item base的协同过滤模型
# 直接通过id-mat二维矩阵计算相似度
from sklearn.metrics.pairwise import pairwise_distances
class BaseCF:
    def __init__(self,id_mat_df):
        self.id_mat_df = id_mat_df
        self.id_mat_matrix = id_mat_df.to_numpy(dtype=np.float64)
        self.mat_similarity = None
        self.sup_similarity = None
    def conduct_similarity(self):
        self.mat_similarity = pairwise_distances(self.id_mat_matrix.T)
        # self.sup_similarity = pairwise_distances(self.id_mat_matrix)
        return
    def get_sup(self, mat):
        sup_ids = []
        try:
            sup_ids = list(id_mat_df[id_mat_df.get(mat_name)>0].index)
        except:
            pass
        return sup_ids
    def get_neareast_mat(self, mat, n=10):
        mat_indx = id_mat_df.columns.get_indexer([mat])[0]
        similar_mat_indx = self.mat_similarity[mat_indx].argsort()[:n]
        return list(self.id_mat_df.columns[similar_mat_indx])
    def get_neareast_supplier(self, mat, simMatNum=10, n=100):
        neareast_mat = self.get_neareast_mat(mat, n=simMatNum)
        visited = set()
        sup_id = []
        for mat in neareast_mat:
            suppliers = list(self.id_mat_df[self.id_mat_df[mat]>0].index)
            for id in suppliers:
                if id not in visited:
                    sup_id.append(id)
                    visited.add(id)
        return list(sup_id)[:n]

In [5]:
# np.save("./mat_similarity1.npy",baseCF.mat_similarity)
mat_similarity = np.load("./mat_similarity1.npy")

In [6]:
baseCF = BaseCF(id_mat_df)
baseCF.mat_similarity = mat_similarity
# baseCF.conduct_similarity()

In [9]:
# np.save("./mat_similarity1.npy",baseCF.mat_similarity)

In [ ]:
def getMatVec(id_mat_df, matName):
    return np.array(id_mat_df[matName]).T

In [22]:
print(getMatVec(id_mat_df,["轴承","调心滚子轴承","圆珠笔","坐便器"]))

[[0 4 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [20]:
mat = '排气管'
baseCF.get_neareast_mat(mat,n = 20)

['汽油传感器',
 '充电机',
 '主刷',
 '实心轮胎',
 '后主钢板弹簧第一片',
 '门边框',
 '后轴承',
 '排气管',
 '变矩器芯',
 '柴油油浮子',
 '变矩操控阀',
 '前轮毂',
 '预热塞',
 '点火线圈',
 '铲头螺杆',
 '废气阀',
 '柴油滤清器(国三)',
 '助力泵',
 '雨刮总成',
 '车辆行车记录仪']

In [14]:
supp = baseCF.get_neareast_supplier(mat, simMatNum=100, n=50)
print(supp)
for id in supp:
    print(id_mat_df.loc[id][id_mat_df.loc[id]>0][:10].index)

[6878, 10198, 19096, 27053, 32287, 13506, 42365, 18643, 19060, 34908, 24267, 38055, 4877, 8167, 13770, 251, 23601, 39618, 17216, 38012, 5634, 32914, 8567, 39432, 18652, 7702, 8974, 14779, 16630, 33183, 16128, 31399, 10184, 4551, 25546, 8570, 8631, 12668, 44673, 19315, 40417, 34314, 525, 31840, 8418, 39236, 25147, 23734, 10211, 23694]
Index(['下水软管', '水箱洁具', '荧光灯管', '脚踏阀弹簧', '小便池下水洁具', '便携式测振仪', '拖把池', '电子维修组合工具',
       '工具箱', '蓬布'],
      dtype='object')
Index(['下水软管', '水箱洁具', '泡沫胶', '报警器', '荧光灯管', '磁性物电流板', '磁性物振荡板', '脚踏阀弹簧',
       '小便池下水洁具', '便携式测振仪'],
      dtype='object')
Index(['描图纸', '下水软管', '水箱洁具', '帆布棉门帘', '透明胶布', '反光牌板', '双舌门锁', '磁性白板', '平板拖布',
       '热敏打印纸'],
      dtype='object')
Index(['下水软管', '水箱洁具', '泡沫胶', '报警器', '荧光灯管', '脚踏阀弹簧', '小便池下水洁具', '稳压电源', '拖把池',
       '彩条布'],
      dtype='object')
Index(['描图纸', '下水软管', '聚合氯化铝铁', '水箱洁具', '金属套玻璃温度计', '透明胶布', '磁性白板', '热敏打印纸',
       '荧光灯管', '脚踏阀弹簧'],
      dtype='object')
Index(['聚合氯化铝铁', '泡沫胶', '彩条布', '电子维修组合工具', '工具箱', '蓬布', '

In [ ]:
mat_indx = id_mat_df.columns.get_indexer([mat])[0]
similar_mat_indx = mat_similarity[mat_indx].argsort()[:100]
list(id_mat_df.columns[similar_mat_indx])

['汽油传感器',
 '充电机',
 '主刷',
 '实心轮胎',
 '后主钢板弹簧第一片',
 '门边框',
 '后轴承',
 '排气管',
 '变矩器芯',
 '柴油油浮子',
 '变矩操控阀',
 '前轮毂',
 '预热塞',
 '点火线圈',
 '铲头螺杆',
 '废气阀',
 '柴油滤清器(国三)',
 '助力泵',
 '雨刮总成',
 '车辆行车记录仪',
 '电子扇',
 '主刷电机',
 '刹车鼓',
 '轮边油封',
 '进油管',
 '\t取力器进油管',
 '手扶箱总成',
 '刹车蹄总成',
 '前轴头',
 '门外把手',
 '行星齿轮垫片',
 '下摆臂',
 '副刷',
 '链条齿轮',
 '三元催化器',
 '离合器分离轴承总成',
 '驾驶室安全带',
 '后主钢板中心螺栓',
 '暖风水管',
 '机油滤清器(国三)',
 '转向立轴',
 '可变电阻',
 '爆震传感器',
 '加长货叉',
 '前轮毂外轴承总成',
 '左前玻璃升降器开关',
 '气门挺住',
 '真空助力泵',
 '前叉盘',
 '全车垫',
 '压力阀',
 '前碟刹片',
 '真空助力器',
 '\t方向机修理包',
 '气门室盖总成',
 '后桥骑马螺栓',
 '发动机支架',
 '线束',
 '加油枪油管',
 '充电连接器',
 '前叶子板右',
 '下视镜',
 '前轮毂内轴承总成',
 '汽油泵',
 '风扇叶子',
 '前轮胎螺栓',
 '转向节',
 '空调滤芯',
 '左前照灯总成',
 '启动机',
 '门锁总成',
 '液压传动油',
 '钢圈',
 '危化品卸车安全确认表（综合罐区）',
 '裁纸手工刀',
 '铁皮柜',
 '地上式消防栓',
 '壁挂式扬声器',
 '充电保险',
 '烟雾检测智能球型摄像机',
 '300ZJ-1-A70（64.5）填料压盖',
 '省力型尖嘴钳',
 '芯棒轮驱动轴承',
 '二棒修复件机加工结算（零星紧急返修）',
 '后焊接制动蹄总成加宽',
 '轴承 BEARING CYR  1 1/2 S CYR  1 1/2 SW',
 '精矿粉',
 'C200鄂破机\\螺母N01563299',
 '氢 氟 酸',
 '六色机爬坡皮带驱动轴',
 '夹紧角钢',
 '右侧导流板安装支架',
 '

In [3]:
pd.DataFrame([[1,2],[3]]).to_csv("a.csv")